<a href="https://colab.research.google.com/github/joseluisll/AnalisisIbex/blob/main/AnalisisIbex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# Instala plotly en Colab
!pip install plotly

import numpy as np
import pandas as pd
import plotly.graph_objects as go

**Creamos el vector de tiempos, para todos el mismo, cada posicion es un día.**

In [51]:
vector_fechas = pd.date_range(start='2015-01-01', end='2025-04-14')

#Preparar la serie HISTORICA de IBEX35 desde 1/1/2015 A 11/04/2025

In [65]:
# Abrir la serie de IBEX35
df_ibex = pd.read_csv('datos/datos_historicos_IBEX35.csv', parse_dates=['Fecha'], index_col='Fecha')
# Convert 'Último' column to float
df_ibex['Último'] = df_ibex['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))


In [66]:
# Crear un dataframe con índice vector_fechas y valores en cero
df_ibex_reindexado = pd.DataFrame(index=vector_fechas, data=0, columns=df_ibex.columns)
df_ibex_reindexado = df_ibex_reindexado.astype(object)

# Copiar los valores de df_ibex a df_ibex_reindexado cuando los índices coinciden
for fecha in df_ibex.index:
  if fecha in df_ibex_reindexado.index:
    df_ibex_reindexado.loc[fecha] = df_ibex.loc[fecha]



# Method 1: Min-Max Scaling (Normalization to [0, 1])
df_ibex_reindexado['Normalizado'] = (df_ibex_reindexado['Último'] - df_ibex_reindexado['Último'].min()) / (df_ibex_reindexado['Último'].max() - df_ibex_reindexado['Último'].min())

#Preparar la serie HISTORICA de Petroleo_WTI desde 1/1/2015 A 11/04/2025

In [67]:
#datos históricos de petroleo_WTI y la normalizo
df_petroleo = pd.read_csv('datos/datos_historico_petroleo_wti.csv', parse_dates=['Fecha'], index_col='Fecha')




In [68]:
# Convert 'Último' column to float
df_petroleo['Último'] = df_petroleo['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

In [69]:
df_petroleo_reindexado = pd.DataFrame(index=vector_fechas, data=0, columns=df_petroleo.columns)
df_petroleo_reindexado = df_petroleo_reindexado.astype(object)

# Copiar los valores de df_petroleo a df_petroleo_reindexado cuando los índices coinciden
for fecha in df_petroleo.index:
  if fecha in df_petroleo_reindexado.index:
    df_petroleo_reindexado.loc[fecha] = df_petroleo.loc[fecha]


# Min-Max Scaling
df_petroleo_reindexado['Normalizado'] = (df_petroleo_reindexado['Último'] - df_petroleo_reindexado['Último'].min()) / (df_petroleo_reindexado['Último'].max() - df_petroleo_reindexado['Último'].min())


**PREPARAR LA SERIE HISTÓRICA DE VOLATILIDAD DEL VIX MISMA ESCALA TEMPORAL QUE LAS ANTERIORES**

In [78]:
# Abrir la serie de IBEX35
df_vix = pd.read_csv('datos/datos_historicos_VIX.csv', parse_dates=['Fecha'], index_col='Fecha')

# Convert 'Último' column to float
df_vix['Último'] = df_vix['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

df_vix_reindexado = pd.DataFrame(index=vector_fechas, data=0, columns=df_vix.columns)
df_vix_reindexado = df_vix_reindexado.astype(object)

# Copiar los valores de df_petroleo a df_petroleo_reindexado cuando los índices coinciden
for fecha in df_vix.index:
  if fecha in df_petroleo_reindexado.index:
    df_vix_reindexado.loc[fecha] = df_vix.loc[fecha]


# Min-Max Scaling
df_vix_reindexado['Normalizado'] = (df_vix_reindexado['Último'] - df_vix_reindexado['Último'].min()) / (df_vix_reindexado['Último'].max() - df_vix_reindexado['Último'].min())


#REPRESENTAR LAS SERIES TEMPORALES NORMALIZADAS SOBRE EL MISMO EJE

El grafico es interactivo y permite hacer zoom.

El OJO humano y el cerebro ya ha visto que hay relación entre ambas series.

In [84]:
# 1. Select 'Último' column from both DataFrames
df_ibex_reindexado['Derivada'] = df_ibex_reindexado['Normalizado'].diff()

ibex_ultimo = df_ibex_reindexado['Derivada']
df_petroleo_reindexado['Derivada'] = df_petroleo_reindexado['Normalizado'].diff()

petroleo_ultimo = df_petroleo_reindexado['Normalizado']
df_vix_reindexado['Derivada'] = df_vix_reindexado['Normalizado'].diff()

vix_ultimo = df_vix_reindexado['Derivada']




# Crear DataFrame para Plotly
df_to_plot = pd.DataFrame({'IBEX35': ibex_ultimo, 'Petroleo_WTI': petroleo_ultimo, 'VIX' : vix_ultimo},index=vector_fechas)
print(len(ibex_ultimo))
print(len(petroleo_ultimo))
print(len(df_vix_reindexado))


# Crear gráfico interactivo
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_to_plot.index, y=df_to_plot['IBEX35'],
                         mode='lines', name='IBEX35'))

fig.add_trace(go.Scatter(x=df_to_plot.index, y=df_to_plot['Petroleo_WTI'],
                         mode='lines', name='Petroleo_WTI'))

fig.add_trace(go.Scatter(x=df_to_plot.index, y=df_to_plot['VIX'],
                         mode='lines', name='VIX'))
# Actualizar diseño del gráfico
fig.update_layout(title='IBEX35 vs Petroleo_WTI vs VIX',
                  xaxis_title='Fecha',
                  yaxis_title='Valor',
                  hovermode='x unified',
                  template='plotly_white')

fig.show()


3757
3757
3757


Yo concluiría que el índice de petroleo_WTI puede ser un indice adelantado del IBEX 35.

¿Cómo puedo calcular el tiempo de adelanto?
¿Habrá un factor de correlación?
¿Esto se podría hacer con un modelo?.
Problemas con los ejes, tienen diferente longitud, hay que ajustar siempre al eje con la misma longitud.

In [72]:
# Assuming df_to_plot is already defined and contains your data

# Set the window size for the rolling correlation
window_size = 30  # You can adjust this to your needs

# Calculate the rolling correlation
rolling_correlation = df_to_plot['IBEX35'].rolling(window=window_size).corr(df_to_plot['Petroleo_WTI'])

# Create a new DataFrame for the rolling correlation
df_correlation = pd.DataFrame({'Correlation': rolling_correlation})
df_correlation = df_correlation.reindex(df_to_plot.index, method='pad')

# You can now plot or analyze df_correlation

In [73]:
# Add trace for the correlation
fig.add_trace(go.Scatter(x=df_correlation.index, y=df_correlation['Correlation'],
                         mode='lines', name='Correlation', yaxis='y2'))

# Update layout for the second y-axis
fig.update_layout(
    yaxis2=dict(
        title='Correlation',
        overlaying='y',
        side='right',
        range=[-1, 1]  # Set range for correlation
    ),
    title='IBEX35, Petroleo_WTI, and Correlation',
    xaxis_title='Fecha',
    yaxis_title='Valor',
    hovermode='x unified',
    template='plotly_white'
)

# Show the plot

In [74]:
# Assuming df_to_plot is already defined

# Calculate the cross-correlation
cross_corr = np.correlate(df_to_plot['IBEX35'], df_to_plot['Petroleo_WTI'], mode='full')

# Find the lag with the maximum correlation
lags = np.arange(-len(df_to_plot) + 1, len(df_to_plot))
lag_max = lags[np.argmax(cross_corr)]

print(f"Estimated temporal lead of Petroleo_WTI over IBEX35: {lag_max} days")

Estimated temporal lead of Petroleo_WTI over IBEX35: 0 days
